In [1]:
from tensorflow import keras
from tensorflow.keras import layers

### 사용자 정의 지표 만들기

**`Metric` 클래스를 상속하여 사용자 정의 지표 구현하기**

In [2]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [3]:
from tensorflow.keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [4]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()]) # Custom Metric is used
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2940 - accuracy: 0.9129 - rmse: 7.1860 - val_loss: 0.1477 - val_accuracy: 0.9576 - val_rmse: 7.3485
Epoch 2/3
1563/1563 [==============================] - 10s 7ms/step - loss: 0.1627 - accuracy: 0.9539 - rmse: 7.3564 - val_loss: 0.1175 - val_accuracy: 0.9679 - val_rmse: 7.4020
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.1002 - accuracy: 0.9731 - rmse: 7.4315


In [8]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", keras.metrics.RootMeanSquaredError()])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2949 - accuracy: 0.9125 - root_mean_squared_error: 5.2357 - val_loss: 0.1499 - val_accuracy: 0.9581 - val_root_mean_squared_error: 5.2180
Epoch 2/3
1563/1563 [==============================] - 10s 7ms/step - loss: 0.1626 - accuracy: 0.9535 - root_mean_squared_error: 5.2365 - val_loss: 0.1295 - val_accuracy: 0.9646 - val_root_mean_squared_error: 5.2181
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.1076 - accuracy: 0.9713 - root_mean_squared_error: 5.2285


In [9]:

Image Data : Activation / Dropout rate
    

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)